In [15]:
# !pip install -q -U tensorflow-text
# !pip install pipwin
# !pipwin install PyAudio
# !pip install SpeechRecognition pydub
# !pip install gtts
# !pip install playsound
!pip install sentence-transformers

  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4


In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from keras.regularizers import l2
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import f1_score
np.random.seed(1)
import tensorflow_hub as hub
import tensorflow_text as text
# from official.nlp import optimization
import matplotlib.pyplot as plt
import re
import string
import seaborn as sns
import speech_recognition as sr #SpeechToText
from gtts import gTTS #TextToSpeech
from playsound import  playsound
import os
tf.get_logger().setLevel('ERROR')

In [2]:
def takeCommandHindi():
         
    r = sr.Recognizer()
    with sr.Microphone() as source:
          
        # seconds of non-speaking audio before 
        # a phrase is considered complete
        print('Listening')
        r.pause_threshold = 0.7  
        audio = r.listen(source)  
        try:
            print("Recognizing")
            Query = r.recognize_google(audio, language='hi-In')
              
            # for listening the command in indian english
#             print("the query is printed='", Query, "'")
          
        # handling the exception, so that assistant can 
        # ask for telling again the command
        except Exception as e:
            print(e)  
            print("Say that again sir")
            return "None"
        print(Query)
        return Query

In [3]:
import json
data_file = open('data.json',encoding='utf-8').read()
intents = json.loads(data_file)

In [4]:
sentence=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        sentence.append(pattern)

In [5]:
classes=[]
for intent in intents['intents']:
  for pattern in intent['patterns']:
    classes.append(intent['tag'])

In [6]:
print(sentence)

['सीज़ेंट्रिक्स कॉलबैक', 'सीजेड कैक्लबैक', 'सीज़ेंट्रिक्स कॉल बैक', 'C केंद्र कॉल बैक', 'सीज़ेंट्रिक्स कॉल बैक', 'वापस कॉल करें', 'क्या आप कॉल बैक सुविधा प्रदान करते हैं', 'कॉल बैक सॉल्यूशन', 'एक कॉल बैक समाधान की तलाश में', 'कॉल बैक सेवा की आवश्यकता है', 'कॉल बैक सॉल्यूशन है', 'सीज़ेंट्रिक्स कॉलबैक क्या है', 'क्या आप मुझे सीज़ेंट्रिक्स कॉलबैक समाधान में मदद कर सकते हैं', 'क्या आप मुझे सीज़ेंट्रिक्स कॉल बैक सॉल्यूशन में मदद कर सकते हैं', 'क्या आप मुझे कॉल बैक समाधान के बारे में बता सकते हैं', 'ठीक है धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताओ', 'धन्यवाद, मुझे कॉलबैक के बारे में बताएं', 'थैंक्यू, कॉल बैक क्या है', 'अच्छा, क्या आप मुझे कॉलबैक के बारे में बता सकते हैं?', 'धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताएं', ' सीज़ेंट्रिक्स हेल्पडेस्क', 'सीज़ेंट्रिक्सहेल्पडेस्क', 'सीज़ेंट्रिक्स मदद', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'मदद', 'सहायता केंद्र', 'हेल्प डेस्क', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'क्या आप हेल्पडेस्क सुविधा प्रदान करते हैं', 'एक हेल्प डेस्क समाधान की तलाश में', 'मदद ड

In [9]:
print(classes)

['cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'no

In [7]:
from numpy import array
from numpy import argmax
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
encoder = LabelEncoder()
encoder.fit(classes)
encoded_Y = encoder.transform(classes)
encoded_Y=np_utils.to_categorical(encoded_Y)

In [8]:
print(encoded_Y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
X=[]
for i in sentence:
  X.append(i)

In [10]:
print(X)

['सीज़ेंट्रिक्स कॉलबैक', 'सीजेड कैक्लबैक', 'सीज़ेंट्रिक्स कॉल बैक', 'C केंद्र कॉल बैक', 'सीज़ेंट्रिक्स कॉल बैक', 'वापस कॉल करें', 'क्या आप कॉल बैक सुविधा प्रदान करते हैं', 'कॉल बैक सॉल्यूशन', 'एक कॉल बैक समाधान की तलाश में', 'कॉल बैक सेवा की आवश्यकता है', 'कॉल बैक सॉल्यूशन है', 'सीज़ेंट्रिक्स कॉलबैक क्या है', 'क्या आप मुझे सीज़ेंट्रिक्स कॉलबैक समाधान में मदद कर सकते हैं', 'क्या आप मुझे सीज़ेंट्रिक्स कॉल बैक सॉल्यूशन में मदद कर सकते हैं', 'क्या आप मुझे कॉल बैक समाधान के बारे में बता सकते हैं', 'ठीक है धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताओ', 'धन्यवाद, मुझे कॉलबैक के बारे में बताएं', 'थैंक्यू, कॉल बैक क्या है', 'अच्छा, क्या आप मुझे कॉलबैक के बारे में बता सकते हैं?', 'धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताएं', ' सीज़ेंट्रिक्स हेल्पडेस्क', 'सीज़ेंट्रिक्सहेल्पडेस्क', 'सीज़ेंट्रिक्स मदद', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'मदद', 'सहायता केंद्र', 'हेल्प डेस्क', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'क्या आप हेल्पडेस्क सुविधा प्रदान करते हैं', 'एक हेल्प डेस्क समाधान की तलाश में', 'मदद ड

In [15]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('google/muril-base-cased')

Some weights of the model checkpoint at C:\Users\lohia/.cache\torch\sentence_transformers\google_muril-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
sentence_embedding=sbert_model.encode(X)

C:\Users\lohia\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [18]:
model = Sequential()
model.add(Dense(256, input_shape=(sentence_embedding.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(33, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
hist = model.fit(sentence_embedding,encoded_Y, epochs=300, batch_size=10, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/300
53/53 [==============================] - 1s 2ms/step - loss: 3.4976 - accuracy: 0.0302
Epoch 2/300
53/53 [==============================] - 0s 2ms/step - loss: 3.4721 - accuracy: 0.0472
Epoch 3/300
53/53 [==============================] - 0s 2ms/step - loss: 3.4551 - accuracy: 0.0528
Epoch 4/300
53/53 [==============================] - 0s 2ms/step - loss: 3.4407 - accuracy: 0.0604
Epoch 5/300
53/53 [==============================] - 0s 2ms/step - loss: 3.4366 - accuracy: 0.0566
Epoch 6/300
53/53 [==============================] - 0s 2ms/step - loss: 3.4178 - accuracy: 0.0755
Epoch 7/300
53/53 [==============================] - 0s 2ms/step - loss: 3.3999 - accuracy: 0.0755
Epoch 8/300
53/53 [==============================] - 0s 2ms/step - loss: 3.3822 - accuracy: 0.0830
Epoch 9/300
53/53 [==============================] - 0s 2ms/step - loss: 3.3507 - accuracy: 0.0811
Epoch 10/300
53/53 [==============================] - 0s 2ms/step - loss: 3.3148 - accuracy: 0.0811
Epoch 11/

53/53 [==============================] - 0s 2ms/step - loss: 0.9892 - accuracy: 0.6830
Epoch 83/300
53/53 [==============================] - 0s 2ms/step - loss: 0.9555 - accuracy: 0.6868
Epoch 84/300
53/53 [==============================] - 0s 2ms/step - loss: 0.9211 - accuracy: 0.6962
Epoch 85/300
53/53 [==============================] - 0s 2ms/step - loss: 0.9575 - accuracy: 0.6774
Epoch 86/300
53/53 [==============================] - 0s 2ms/step - loss: 0.9115 - accuracy: 0.6830
Epoch 87/300
53/53 [==============================] - 0s 2ms/step - loss: 0.8853 - accuracy: 0.6981
Epoch 88/300
53/53 [==============================] - 0s 2ms/step - loss: 0.9023 - accuracy: 0.7170
Epoch 89/300
53/53 [==============================] - 0s 2ms/step - loss: 0.8599 - accuracy: 0.7038
Epoch 90/300
53/53 [==============================] - 0s 2ms/step - loss: 0.8689 - accuracy: 0.6849
Epoch 91/300
53/53 [==============================] - 0s 2ms/step - loss: 0.8965 - accuracy: 0.7019
Epoch 92/300


53/53 [==============================] - 0s 2ms/step - loss: 0.3563 - accuracy: 0.8925
Epoch 164/300
53/53 [==============================] - 0s 2ms/step - loss: 0.4187 - accuracy: 0.8453
Epoch 165/300
53/53 [==============================] - 0s 2ms/step - loss: 0.3691 - accuracy: 0.8623
Epoch 166/300
53/53 [==============================] - 0s 2ms/step - loss: 0.3641 - accuracy: 0.8774
Epoch 167/300
53/53 [==============================] - 0s 3ms/step - loss: 0.3964 - accuracy: 0.8566
Epoch 168/300
53/53 [==============================] - 0s 2ms/step - loss: 0.3725 - accuracy: 0.8642
Epoch 169/300
53/53 [==============================] - 0s 2ms/step - loss: 0.3463 - accuracy: 0.8830
Epoch 170/300
53/53 [==============================] - 0s 2ms/step - loss: 0.3417 - accuracy: 0.8774
Epoch 171/300
53/53 [==============================] - 0s 2ms/step - loss: 0.3672 - accuracy: 0.8717
Epoch 172/300
53/53 [==============================] - 0s 2ms/step - loss: 0.3239 - accuracy: 0.9057
Epoc

53/53 [==============================] - 0s 2ms/step - loss: 0.1783 - accuracy: 0.9434
Epoch 245/300
53/53 [==============================] - 0s 2ms/step - loss: 0.1915 - accuracy: 0.9264
Epoch 246/300
53/53 [==============================] - 0s 2ms/step - loss: 0.2017 - accuracy: 0.9226
Epoch 247/300
53/53 [==============================] - 0s 2ms/step - loss: 0.1694 - accuracy: 0.9340
Epoch 248/300
53/53 [==============================] - 0s 2ms/step - loss: 0.2145 - accuracy: 0.9208
Epoch 249/300
53/53 [==============================] - 0s 2ms/step - loss: 0.1944 - accuracy: 0.9264
Epoch 250/300
53/53 [==============================] - 0s 2ms/step - loss: 0.2022 - accuracy: 0.9340
Epoch 251/300
53/53 [==============================] - 0s 2ms/step - loss: 0.2101 - accuracy: 0.9245
Epoch 252/300
53/53 [==============================] - 0s 2ms/step - loss: 0.1851 - accuracy: 0.9415
Epoch 253/300
53/53 [==============================] - 0s 2ms/step - loss: 0.1982 - accuracy: 0.9170
Epoc

In [11]:
import random
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [12]:
final=['','affirm','crm_application','cz_acd','cz_bar','cz_bot','cz_callback',
 'cz_callblast','cz_chat','cz_conference','cz_dialer','cz_guide',
 'cz_helpdesk','cz_ivr','cz_lms','cz_magic_call','cz_missed_call',
 'cz_mobile','cz_screen_logger','cz_voice_logger','deny','greet','home',
 'manual_dialer','non_voice_process','predictive_dialer','preview_dialer',
 'product','progressive_dialer','self_service_solutions','thankyou',
 'voice_process','work_force_management']

In [13]:
def chatbot_response():
  text= takeCommandHindi()  
  text=[text]
  em=sbert_model.encode(text)
  res = model.predict(em)[0]
  ERROR_THRESHOLD = 0.3
  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
      return_list.append({"intent": final[r[0]], "probability": str(r[1])})
  res = getResponse(return_list, intents)
  return res
 

In [45]:
res = chatbot_response()
myobj=gTTS(text=res,lang='hi',slow=True)
myobj.save("answer.mp3")
audio_file = "D:\\\\answer.mp3"
print(audio_file)
playsound(audio_file)
os.remove(audio_file)

Listening
Recognizing
नमस्ते


C:\Users\lohia\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
ERROR:playsound:
    Error 305 for command:
        open "D:\\answer.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.
ERROR:playsound:
    Error 263 for command:
        close "D:\\answer.mp3"
    The specified device is not open or is not recognized by MCI.


D:\\answer.mp3


PlaysoundException: 
    Error 305 for command:
        open "D:\\answer.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.

In [43]:
chatbot_response("सीज़ेंट्रिक्स कॉलबैक")

TypeError: chatbot_response() takes 0 positional arguments but 1 was given

In [ ]:
chatbot_response("ठीक है धन्यवाद मुझे सीआरएम आवेदन के बारे में बताओ")

सीज़ेंट्रिक्स के पास ये हैं सीज़ेंट्रिक्स बार सीज़ेंट्रिक्स एलएमएस सीज़ेंट्रिक्स हेल्पडेस्क


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
chatbot_response("kya aapke paas progressive mode hai")

सीज़ेंट्रिक्स कॉन्फ़्रेंस स्वचालित डायलर, अनुस्मारक सूचनाएं और रिकॉर्ड मीटिंग जैसी कई उन्नत सुविधाएँ प्रदान करता है। सभी कॉल पासवर्ड से सुरक्षित हैं। एसएमएस और ईमेल एकीकरण कुशल अधिसूचना क्षमता प्रदान करता है। अधिक जानकारी के लिए यात्रा लिंक,<a href='https://www.c-zentrix.com/products/cz-conferencing-bridge'>यहाँ क्लिक करें</a>


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
chatbot_response("हांजी")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


आपकी प्रतिक्रिया के लिए आपका धन्यवाद। होम पर जाए
